In [118]:
# Importing Libraries
import numpy as np
import torch
from torch import nn
import skorch
from skorch import NeuralNetRegressor as SkorchNNR
from modAL.models import BayesianOptimizer
from modAL.acquisition import max_PI
import numpy as np

# Generating Dataset
row,col = 50,2048
X = np.random.randint(2, size=(row,col)).astype(np.float32)
y = np.random.randn(row,1).astype(np.float32) ** 2

# Picking out initial training samples
train_idx = np.random.choice(range(X.shape[0]), size=int(X.shape[0]*0.2), replace=False)

x_initial = X[train_idx]
y_initial = y[train_idx]

X = np.delete(X,train_idx, axis=0)
y = np.delete(y,train_idx)

# Displaying Shape after initial selection
print(X.shape, y.shape)
print(x_initial.shape, y_initial.shape)

# PyTorch Model
class Torch_Model_BasicRegressor(nn.Module):
    def __init__(self):
        super(Torch_Model_BasicRegressor, self).__init__()
        self.fcs = nn.Sequential(
            nn.Linear(2048,256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256,32),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(32,4),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(4,1)
        )
    def forward(self, x):
        out = self.fcs(x)
        return out

# SKORCH model
device = "cuda" if torch.cuda.is_available()==True else "cpu"
regressor = SkorchNNR(
    Torch_Model_BasicRegressor,
    lr=1e-5,
    criterion= nn.MSELoss,
    max_epochs = 25,
    batch_size = 128,
    train_split = skorch.dataset.ValidSplit(5) if int(X.shape[0]*0.2)>5 else skorch.dataset.ValidSplit(int(X.shape[0]*0.2)),
    optimizer =torch.optim.Adam,
    device = device,
    verbose = 1
)

# modAL model
learner = BayesianOptimizer(
    estimator=regressor,
    query_strategy=max_PI,
    X_training=x_initial, y_training=y_initial.reshape(-1,1)
)

# Querying & Teaching the model : Error over here
for n_query in range(15):
    query_idx,query_inst = learner.query(X, n_instances=5)
    learner.teach(X=query_inst,y=y[query_idx])

(40, 2048) (40,)
(10, 2048) (10, 1)
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.2097        0.0911  0.0087
      2        0.2229        0.0908  0.0227
      3        0.1964        0.0909  0.0091
      4        0.1855        0.0911  0.0097
      5        0.2272        0.0915  0.0125
      6        0.2159        0.0918  0.0095
      7        0.2386        0.0922  0.0149
      8        0.1931        0.0926  0.0145
      9        0.1997        0.0931  0.0153
     10        0.2086        0.0934  0.0097
     11        0.1878        0.0937  0.0143
     12        0.2032        0.0942  0.0069
     13        0.2261        0.0945  0.0087
     14        0.2113        0.0947  0.0075
     15        0.2084        0.0949  0.0086
     16        0.1962        0.0954  0.0065
     17        0.1796        0.0957  0.0080
     18        0.1979        0.0958  0.0070
     19        0.2014        0.0960  0.0080
     20        0.2259        0.0960  0.0

TypeError: predict() got an unexpected keyword argument 'return_std'

In [119]:
# Importing Libraries
import numpy as np
import torch
from torch import nn
import skorch
from skorch import NeuralNetRegressor as SkorchNNR
from modAL.models import BayesianOptimizer
from modAL.acquisition import max_PI
import numpy as np

# Generating Dataset
row,col = 50,2048
X = np.random.randint(2, size=(row,col)).astype(np.float32)
y = np.random.randn(row,1).astype(np.float32) ** 2

# Picking out initial training samples
train_idx = np.random.choice(range(X.shape[0]), size=int(X.shape[0]*0.2), replace=False)

x_initial = X[train_idx]
y_initial = y[train_idx]

X = np.delete(X,train_idx, axis=0)
y = np.delete(y,train_idx)

# Displaying Shape after initial selection
print(X.shape, y.shape)
print(x_initial.shape, y_initial.shape)

# PyTorch Model
class Torch_Model_BasicRegressor(nn.Module):
    def __init__(self):
        super(Torch_Model_BasicRegressor, self).__init__()
        self.fcs = nn.Sequential(
            nn.Linear(2048,256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256,32),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(32,4),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(4,1)
        )
    def forward(self, x):
        out = self.fcs(x)
        return out

# SKORCH model
device = "cuda" if torch.cuda.is_available()==True else "cpu"
regressor = SkorchNNR(
    Torch_Model_BasicRegressor,
    lr=1e-5,
    criterion= nn.MSELoss,
    max_epochs = 25,
    batch_size = 128,
    train_split = skorch.dataset.ValidSplit(5) if int(X.shape[0]*0.2)>5 else skorch.dataset.ValidSplit(int(X.shape[0]*0.2)),
    optimizer =torch.optim.Adam,
    device = device,
    verbose = 1
)

# modAL model
learner = BayesianOptimizer(
    estimator=regressor,
    query_strategy=max_PI,
    X_training=x_initial, y_training=y_initial.reshape(-1,1)
)

# Querying & Teaching the model : Error over here
for n_query in range(15):
    query_idx,query_inst = learner.query(X, n_instances=5)
    learner.teach(X=query_inst,y=y[query_idx])

(40, 2048) (40,)
(10, 2048) (10, 1)
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        2.8627        0.6128  0.0140
      2        2.7418        0.6069  0.0150
      3        2.8011        0.6013  0.0100
      4        2.8258        0.5956  0.0110
      5        2.7296        0.5901  0.0110
      6        2.8030        0.5844  0.0090
      7        2.7807        0.5790  0.0090
      8        2.7637        0.5739  0.0090
      9        2.6813        0.5691  0.0080
     10        2.7376        0.5647  0.0090
     11        2.7375        0.5633  0.0110
     12        2.8105        0.5626  0.0090
     13        2.7397        0.5619  0.0080
     14        2.7086        0.5612  0.0090
     15        2.6706        0.5603  0.0130
     16        2.7576        0.5595  0.0130
     17        2.6662        0.5587  0.0120
     18        2.6695        0.5580  0.0090
     19        2.7482        0.5572  0.0090
     20        2.6848        0.5564  0.0

TypeError: predict() got an unexpected keyword argument 'return_std'